<a href="https://colab.research.google.com/github/larissavaladao/colab_curuai/blob/main/curuai_acolite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
#import packages used
import ee
import pandas as pd
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import math
import geemap
import os

In [2]:
#authenticate and initialize google earth engine (also necessary for geemap)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mdLex4l1x5GazClQBphOy2Nu4LzO_U7NxTZIxQo1qec&tc=Q1F6KTTX9fFvcXU95DCy6CiFBkIfWpQnmTHZObsg7IM&cc=jun7rHUg-3NP8mFtnVMt_lzXyl185JqzVC46BPuYoWM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn90cAfhLjxllKYm8FZ5tZU3Jph2l2hsz45YNny-kYadhfdvzPQh-c

Successfully saved authorization token.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Curuai dataset

In [4]:
#import the entire dataset and include ID column
dataset = pd.read_excel('/content/drive/MyDrive/CURUAI_PROCESS/Dataset_CFP.xlsx', sheet_name='data',na_values='NaN').loc[:383]
# dataset = pd.read_excel('Dataset_CFP.xlsx', sheet_name='data',na_values='NaN').loc[:383]


dataset['ID'] = range(len(dataset))
dataset['Local_Time'] = dataset['Local Time']
dataset['TIME_UTC'] = dataset['TIME UTC']
dataset['DEPTH_CLASS'] = dataset['DEPTH CLASS']
dataset['SAMPLE_SITE'] = dataset['SAMPLE SITE']
dataset['WATER_PERIOD'] = dataset['WATER PERIOD']
dataset['TOTAL_DEPTH'] = dataset['TOTAL DEPTH']
dataset['SAMPLING_DEPTH'] = dataset['SAMPLING DEPTH']
dataset['CHLOROPHYLL_A'] = dataset['CHLOROPHYLL A']
dataset['CHLOROPHYLL_B'] = dataset['CHLOROPHYLL B']
dataset['ODO'] = dataset['ODO ']

In [5]:
#copy the dataset and select only the variables of interest
dataset_att = dataset.copy()
dataset_att = dataset_att.loc[:,['ID','DATE', 'LATITUDE','LONGITUDE', 'MISSION']]
dataset_att.head()

,ID,DATE,LATITUDE,LONGITUDE,MISSION
0,0,2013-03-08 00:00:00,-2.25127,-55.14622,I
1,1,2013-03-08 00:00:00,-2.25127,-55.14622,I
2,2,2013-03-09 00:00:00,-2.28422,-55.22023,I
3,3,2013-03-09 00:00:00,-2.19696,-55.29953,I
4,4,2013-03-09 00:00:00,-2.221738,-55.270194,I


In [6]:
dataset_att.groupby(['MISSION'])['MISSION'].count()

MISSION
I       71
II      73
III     74
IV      36
IX      24
V       26
VI      25
VII     28
VIII    27
Name: MISSION, dtype: int64

In [7]:
#vizualize variables included in the dataset
dataset_att.columns

Index(['ID', 'DATE', 'LATITUDE', 'LONGITUDE', 'MISSION'], dtype='object')

In [8]:
#transform dataframe in a geodataframe (geometry column with point location)
gdf = gpd.GeoDataFrame(
    dataset_att, geometry=gpd.points_from_xy(dataset_att.LONGITUDE, dataset_att.LATITUDE),
    crs="EPSG:4326"
)
gdf.head()

,ID,DATE,LATITUDE,LONGITUDE,MISSION,geometry
0,0,2013-03-08 00:00:00,-2.25127,-55.14622,I,POINT (-55.14622 -2.25127)
1,1,2013-03-08 00:00:00,-2.25127,-55.14622,I,POINT (-55.14622 -2.25127)
2,2,2013-03-09 00:00:00,-2.28422,-55.22023,I,POINT (-55.22023 -2.28422)
3,3,2013-03-09 00:00:00,-2.19696,-55.29953,I,POINT (-55.29953 -2.19696)
4,4,2013-03-09 00:00:00,-2.221738,-55.270194,I,POINT (-55.27019 -2.22174)


In [9]:
#transform date and time columns in string - necessary to convert to JSON
gdf['DATE'] = gdf['DATE'].astype("str")

In [10]:
# gdf.to_file("/content/drive/MyDrive/CURUAI_PROCESS/points_curuai.shp")

In [11]:
##Convert geodataframe to json - necessary to be read in GEE
dataset_json = gdf.to_json()

In [12]:
#load and select the features of the json data
data_points = json.loads(dataset_json)
data_points = data_points['features']
# data_points

In [13]:
##transform json in in gee object = feature collection
roi_points = ee.FeatureCollection(data_points)
print(roi_points.size().getInfo())

384


In [14]:
#function to insert a property with date of the point - Gee date format
def insert_date (feat):
    return feat.set('system:time_start',ee.Date.parse('YYYY-MM-dd HH:mm:ss',feat.get('DATE')))

In [15]:
#apply date function - new system:time_start property
roi_points = roi_points.map(insert_date)
print(roi_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-55.14622, -2.25127]}, 'id': '0', 'properties': {'DATE': '2013-03-08 00:00:00', 'ID': 0, 'LATITUDE': -2.25127, 'LONGITUDE': -55.14622, 'MISSION': 'I', 'system:time_start': {'type': 'Date', 'value': 1362700800000}}}


In [16]:
#create a polygon around the floodpalin area - roi
roi_poly = ee.FeatureCollection('projects/ee-curuai/assets/roi_poly_curuai')

# Import GEE images

## Define date ranges for filtering images

In [17]:
def date_mission(mission,time):
  return ee.Date(roi_points.filter(ee.Filter.eq('MISSION',mission)).sort('system:time_start',time).first().get('system:time_start'))

In [18]:
mission = 'VI'
advance = 17

In [19]:
if mission == "I":
  col = 1
elif mission == "II":
  col = 2
elif mission == "III":
  col = 3
elif mission == "IV":
  col = 4
elif mission == "V":
  col = 5
elif mission == "VI":
  col = 6
elif mission == "VII":
  col = 7
elif mission == "VIII":
  col = 8
elif mission == "IX":
  col = 9
else:
  print('Mission value error')

In [20]:
#define initial and final date to filter the image collection based on the
#field points date
initial_date = date_mission(mission,True).advance(-advance, 'day')
end_date = date_mission(mission,False).advance(advance, 'day')

print('Data inicial de coleta: ',initial_date.format().getInfo())
print('Data final de coleta: ',end_date.format().getInfo())

Data inicial de coleta:  2016-03-26T00:00:00
Data final de coleta:  2016-05-08T00:00:00


## Import image collections and apply filters and cloud masks

###Landsat 7 collection 2 tier 1 TOA

In [21]:
#Landsat 7 collection 2 tier 1 TOA
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds
landsat7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA")\
            .filterBounds(roi_poly)\
            .filterDate(initial_date,end_date)
# print(ee.Date(landsat7.first().get('system:time_start')).format().getInfo())
# print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [22]:
landsat7.aggregate_array('LANDSAT_PRODUCT_ID')

importar coleção com correção ACOLITE

In [23]:
ld7_path = 'projects/ee-curuai/assets/mission_'+str(mission)+'/LD7_mission'+str(col)
landsat7_acollite = ee.ImageCollection(ld7_path)
landsat7_acollite

In [24]:
landsat7_acollite.aggregate_array('LANDSAT_PRODUCT_ID')

In [25]:
landsat7_acollite.first()

###Landsat 8 collection 2 tier 1 TOA

In [26]:
#Landsat 8 collection 2 tier 1 TOA
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
            .filterBounds(roi_poly)\
            .filterDate(initial_date,end_date)
# print(ee.Date(landsat8.first().get('system:time_start')).format().getInfo())
# print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [27]:
landsat8.aggregate_array('LANDSAT_PRODUCT_ID')

importar coleção com correção ACOLITE

In [28]:
ld8_path = 'projects/ee-curuai/assets/mission_'+str(mission)+'/LD8_mission'+str(col)
landsat8_acollite = ee.ImageCollection(ld8_path)
landsat8_acollite

In [29]:
landsat8_acollite.aggregate_array('LANDSAT_PRODUCT_ID')

In [30]:
landsat8_acollite.first()

###Sentinel 2 level 1C harmonized

In [31]:
def set_orbit_S2(sentinel2Img):
  orbit = ee.String(sentinel2Img.get("PRODUCT_ID")).slice(-22,-16)
  return sentinel2Img.set("Orbit",orbit)

def set_date_S2(sentinel2Img):
  date = ee.String(sentinel2Img.get("PRODUCT_ID")).slice(11,26)
  return sentinel2Img.set("date_S2",date)

def set_orbit_acollite(acollite_img):
  orbit_array = ee.String(acollite_img.get('PRODUCT_ID')).slice(0,6)
  return acollite_img.set("Orbit",orbit_array)

def set_date_acollite(acollite_img):
  date_array = ee.String(acollite_img.get('PRODUCT_ID')).slice(7)
  return acollite_img.set("date_S2",date_array)

In [32]:
#Sentinel 2 level 1C harmonized
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds

s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
            .filterBounds(roi_poly)\
            .filterDate(initial_date,end_date)

sentinel2 = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(roi_poly)\
            .filterDate(initial_date,end_date)


s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(primary=sentinel2,
  secondary=s2Clouds,condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

In [33]:
sentinel2 = ee.ImageCollection(s2SrWithCloudMask).map(set_orbit_S2).map(set_date_S2)

sentinel2.first()

In [34]:
sentinel2.aggregate_array('PRODUCT_ID')

importar coleção com correção ACOLITE

In [35]:
s2_path = 'projects/ee-curuai/assets/mission_'+str(mission)+'/sentinel2_mission'+str(col)
sentinel2_acollite = ee.ImageCollection(s2_path).map(set_orbit_acollite).map(set_date_acollite)
sentinel2_acollite

In [36]:
sentinel2_acollite.aggregate_array('PRODUCT_ID')

In [37]:
sentinel2_acollite.first()

# Preprocessing

## Mask Clouds

### Cloud Masking functions

Sentinel 2 mask

In [74]:
#define cloud masking functions
#VER NOVA MASCARA DE NUVENS DO EE
MAX_CLOUD_PROBABILITY = 20

def maskS2_1C(image):
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return image.updateMask(isNotCloud).copyProperties(image,['system:time_start',"CLOUDY_PIXEL_PERCENTAGE","PRODUCT_ID"])

'''
 * Function to mask clouds using the Sentinel-2 QA band
 * @param {ee.Image} image Sentinel-2 image
 * @return {ee.Image} cloud masked Sentinel-2 image
'''
def maskS2(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).copyProperties(image,['system:time_start',"CLOUDY_PIXEL_PERCENTAGE","PRODUCT_ID"])


Landsat mask

In [39]:
#Based on [ee.Algorithms.Landsat.simpleCloudScore](https://developers.google.com/earth-engine/landsat)
#This function even calculates percentage of cloud  coverage over a particular region. Useful for screening too cloudy images
def scoreL8_TOA (image):
  #Add a cloud score band.  It is automatically called 'cloud'.
  scored = ee.Algorithms.Landsat.simpleCloudScore(image)
  #Create a mask from the cloud score and combine it with the image mask.
  mask = scored.select(['cloud']).lte(MAX_CLOUD_PROBABILITY);
  #Apply the mask to the image and display the result.
  return image.updateMask(mask).copyProperties(image,['system:time_start',"CLOUD_COVER",'LANDSAT_PRODUCT_ID'])
#######################################################################
# Function to mask clouds using the quality band of Landsat 8.
def maskL8 (image):
    qa = image.select('QA_PIXEL')
  # Check that the cloud bit is off.
  # See https://www.usgs.gov/media/files/landsat-8-9-olitirs-collection-2-level-1-data-format-control-book
    mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(mask).copyProperties(image,['system:time_start',"CLOUD_COVER",'LANDSAT_PRODUCT_ID'])

Combine the correted and TOA collections

In [48]:
#join acolite and Gee TOA collections
def combine_img(collection):
  def wrap (img):
    return img.addBands(collection.filter(ee.Filter.eq('LANDSAT_PRODUCT_ID',img.get('LANDSAT_PRODUCT_ID'))).first())
  return wrap

#join acolite and Gee TOA collections
def combine_imgS2(collection):
  def wrap (img):
    return img.addBands(collection.filter(ee.Filter.eq('Orbit',img.get('Orbit')))\
                        .filter(ee.Filter.eq('date_S2',img.get('date_S2'))).first())
  return wrap

### Landsat 7

In [41]:
landsat7_filter = landsat7.filter(ee.Filter.inList('LANDSAT_PRODUCT_ID', landsat7_acollite.aggregate_array('LANDSAT_PRODUCT_ID')))
landsat7_filter

In [42]:
ld7_combine = ee.ImageCollection(landsat7_filter.map(combine_img(landsat7_acollite)))\
          .map(scoreL8_TOA)\
          .map(maskL8)
ld7_combine

### Landsat 8

In [51]:
landsat8_filter = landsat8.filter(ee.Filter.inList('LANDSAT_PRODUCT_ID', landsat8_acollite.aggregate_array('LANDSAT_PRODUCT_ID')))
landsat8_filter

In [52]:
ld8_combine = ee.ImageCollection(landsat8_filter.map(combine_img(landsat8_acollite)))\
          .map(scoreL8_TOA)\
          .map(maskL8)
ld8_combine

### Sentinel 2

In [62]:
sentinel2_filter = sentinel2.filter(ee.Filter.inList('Orbit', sentinel2_acollite.aggregate_array('Orbit')))\
                    .filter(ee.Filter.inList('date_S2', sentinel2_acollite.aggregate_array('date_S2')))
sentinel2_filter

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [75]:
sentinel2_combine = ee.ImageCollection(sentinel2_filter.map(combine_img(sentinel2_acollite)))\
          .map(maskS2_1C)\
          .map(maskS2)
sentinel2_combine

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


### visualização das imagens

Primeira imagem com menos nuvens da colação de iamgens disponiveis durante a missão

In [76]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)

Map.addLayer(ld8_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00002}, str(ld8_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld8_combine.first(), {'bands':['B4','B3','B2'], min:0,max:0.00002}, "TOA"+str(ld8_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld7_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00001}, str(ld7_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(ld7_combine.first(), {'bands':['B3','B2','B1'], min:0,max:0.00002}, "TOA"+str(ld7_combine.first().get('LANDSAT_PRODUCT_ID').getInfo()))
Map.addLayer(sentinel2_combine.first(), {'bands':['b3','b2','b1'], min:0,max:0.00001}, str(sentinel2_combine.first().get('PRODUCT_ID').getInfo()))
Map.addLayer(sentinel2_combine.first(), {'bands':['B4','B3','B2'], min:0,max:0.00002}, "TOA"+str(sentinel2_combine.first().get('PRODUCT_ID').getInfo()))

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

## Padronize band names and projection

In [77]:
#projeção
prj = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_228061_20161111').select(['B4']).projection()
prj

###Landsat 7

In [78]:
#renomear bandas
name_bands = ['blue','green','red','nir','swir1','swir2']
ld7 = ld7_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [79]:
print(ld7.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32621', 'transform': [30, 0, 609900, 0, -30, -219540]}


### Landsat 8

In [80]:
ld8 = ld8_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [81]:
print(ld8.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32621', 'transform': [30, 0, 676170, 0, -30, -219540]}


### Sentinel 2

In [82]:
s2 = sentinel2_combine.map(lambda img: img.select(['b1','b2','b3','b4','b5','b6'])\
                        .rename(name_bands))

In [83]:
print(s2.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32721', 'transform': [10, 0, 609920, 0, -10, 9780460]}


## Transform into remote sensing  reflectance and sunglint correction

In [92]:
#dividir imagem corrigida por pi
# Rrs_sat_ac = Rsat_ac / pi

#fazer deglint
#Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
#correção no artigo INPE CURUAI
def deglint (img):
    Rrs = img.divide(math.pi)
    deglint = Rrs.select(['blue','green','red','nir'])\
    .subtract(Rrs.select('swir1'))

    return (deglint.updateMask(deglint.select(['red']).gt(0))
    .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index","LANDSAT_PRODUCT_ID","PRODUCT_ID","CLOUDY_PIXEL_PERCENTAGE"]))


### Landsat 7

In [93]:
landsat7_rs = ld7.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))

landsat7_rs

### Landsat 8

In [87]:
landsat8_rs = ld8.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))


### Sentinel 2

In [96]:
sentinel2_rs = s2.map(deglint).map(lambda img : img.set({"SATELLITE_PRODUCT_ID":img.get("PRODUCT_ID"),
                                                          "CLOUD_COVER":img.get("CLOUDY_PIXEL_PERCENTAGE")}))

sentinel2_rs

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [132]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
vis =  {'bands':['red','green','blue'],'min':0.01,'max':0.06}
Map.addLayer(landsat7_rs.sort('CLOUD_COVER').first(),vis, 'landsat7')
Map.addLayer(landsat8_rs.sort('CLOUD_COVER').first(),vis, 'landsat8')
Map.addLayer(sentinel2_rs.first(), vis, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

# Sample data points pixel values

filter images that fall within a 16-day window period from each field point date

In [123]:
def imgs_points(collection):
    def wrap(feat):
        date_point = ee.Date(feat.get('system:time_start'))
        data1 = date_point.advance(-16,'day')\
        .format('yyyy-MM-dd')

        data2 = date_point.advance(16,'day')\
        .format('yyyy-MM-dd')

        filtro = collection.filterDate(data1,data2)\
        .filterBounds(feat.geometry())\
        .map(lambda img: img.set({
            'dif_date_point':ee.Date(img.get('system:time_start')).difference(date_point, 'day')})\
            .copyProperties(feat,['ID']))


        return ee.ImageCollection(filtro.limit(10))

    return wrap

obtain statistics for the same location of the field point - with a 3 pixel window - and filter out if more than 4 pixels are masked   

In [133]:
def sample_point(img):
  feat = roi_points.filter(ee.Filter.eq('ID',img.get('ID'))).first()
  geom = feat.geometry().buffer(45).bounds()

  mean = img.reduceRegion(geometry=geom,
                          scale=prj.nominalScale(),
                          crs=prj.crs(),
                          reducer=ee.Reducer.mean())
  median = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.median())
  minMax = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.minMax())
  count = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.count())
  return feat.set({
      "SATELLITE_PRODUCT_ID": img.get('SATELLITE_PRODUCT_ID'),
      'CLOUD_COVER':img.get('CLOUD_COVER'),
      'img_date':ee.Date(img.get('system:time_start')).format(),
      'dif_date_point': img.get('dif_date_point'),
      'blue_mean':mean.get('blue'),
      'green_mean':mean.get('green'),
      'red_mean':mean.get('red'),
      'nir_mean':mean.get('nir'),
      'blue_median':median.get('blue'),
      'green_meadin':median.get('green'),
      'red_median':median.get('red'),
      'nir_median':median.get('nir'),
      'blue_min':minMax.get('blue_min'),
      'green_min':minMax.get('green_min'),
      'red_min':minMax.get('red_min'),
      'nir_min':minMax.get('nir_min'),
      'blue_max':minMax.get('blue_max'),
      'green_max':minMax.get('green_max'),
      'red_max':minMax.get('red_max'),
      'nir_max':minMax.get('nir_max'),
      "count_pixel":count.get('red')
      })

filter_count = ee.Filter.gt('count_pixel',3)




### Landsat 7

In [100]:
img_pointsLD7 = ee.ImageCollection(roi_points.map(imgs_points(landsat7_rs)).flatten().toList(2000))

In [101]:
img_pointsLD7.size()


In [102]:
img_pointsLD7.aggregate_count_distinct('ID')

In [103]:
img_pointsLD7.limit(5)

In [104]:
reduced_LD7 = ee.FeatureCollection(img_pointsLD7.map(sample_point)).filter(filter_count)

In [ ]:
reduced_LD7.limit(2)

In [106]:
reduced_LD7.size()

In [107]:
reduced_LD7.aggregate_count_distinct('ID')

## Landsat 8

In [108]:
img_pointsLD8 = ee.ImageCollection(roi_points.map(imgs_points(landsat8_rs)).flatten().toList(2000))

In [109]:
img_pointsLD8.size()


In [110]:
img_pointsLD8.aggregate_count_distinct('ID')

In [111]:
img_pointsLD8.limit(5)

In [112]:
reduced_LD8 = ee.FeatureCollection(img_pointsLD8.map(sample_point)).filter(filter_count)

In [113]:
reduced_LD8.size()


In [114]:
reduced_LD8.aggregate_count_distinct('ID')

In [115]:
reduced_LD8.limit(2)

##Sentinel 2

In [125]:
img_pointsS2 = ee.ImageCollection(roi_points.map(imgs_points(sentinel2_rs)).flatten().toList(2000))

In [126]:
img_pointsS2.size()

In [127]:
img_pointsS2.aggregate_count_distinct('ID')

In [134]:
reduced_S2 = ee.FeatureCollection(img_pointsS2.map(sample_point)).filter(filter_count)

In [135]:
reduced_S2.size()


In [136]:
reduced_S2.aggregate_count_distinct('ID')

In [131]:
reduced_S2.limit(2)

## Visualize sampled image by defining field point ID

In [137]:
Map = geemap.Map()
Map.centerObject(roi_points,10)
id = 302
vis_params={'bands':['red','green','blue'],'min':0,'max':0.05}
Map.addLayer(img_pointsLD7.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat7')
Map.addLayer(img_pointsLD8.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat8')
Map.addLayer(img_pointsS2.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'sentinel2')

Map.addLayer(roi_points.filter(ee.Filter.eq('ID',id)), {'color':'darkred'}, 'Data Points');
Map

Map(center=[-2.1837863104242503, -55.48976569658883], controls=(WidgetControl(options=['position', 'transparen…

# Export data as CSV file

In [ ]:
#Exportar tabelas
#LD7
geemap.ee_to_csv(reduced_LD7, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD7_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_LD8, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD8_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD8_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_S2, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/VI_S2_DOS_data.csv


CORREÇÂO ATM:


1.   ACOLITE https://github.com/acolite/acolite
2.   6S com imagem da Dinha e parametros do artigo... talvez seja necessaria mais uma correção para isso



